In [1]:
import jpype
import jaydebeapi
import os
import glob
import pandas as pd

# define username
user=os.environ['USER']

# Get hadoop jars
KERBEROS_LOGIN_JAR=r"/san-data/shared/lib/KerberosLogin.jar"
HADOOP_LIB_PATH=os.path.pathsep.join(glob.glob('%s.[jJ][aA][rR]' %
                                               r"/opt/cloudera/parcels/CDH-5.4.3-1.cdh5.4.3.p862.534/jars/*"))

#Start JVM
jpype.startJVM(jpype.getDefaultJVMPath(), "-Djava.class.path=%s:%s" % (KERBEROS_LOGIN_JAR, HADOOP_LIB_PATH))

# Kerberos Login
kerbLoginClass = jpype.JClass('org.statefarm.sr.analytics.dax.hadoop.KerberosLogin')
kerbLoginInstance = kerbLoginClass()
kerbLoginInstance.login(user+"@OPR.STATEFARM.ORG", "/home/"+user+"/"+user+".keytab")

# Establish database connection using jaydebeapi (jaydebeapi will use existing jpype jvm)
conn = jaydebeapi.connect('org.apache.hive.jdbc.HiveDriver',
                          'jdbc:hive2://da74wbedge2.opr.statefarm.org:10000/default;'
                        + 'principal=hive/da74wbedge2.opr.statefarm.org@OPR.STATEFARM.ORG')



In [17]:
%ls

attempt0.ipynb  connecting to hivem.ipynb


In [18]:
os.chdir('/san-data/shared/')
%ls

au/                   fm61/  m9tn_conda/               pyspark/
deap_conda/           fr6v/  metastore_db/             python3/
DEAP - SymReg.ipynb*  kes1/  nltk_data/                spark/
DEAP Tutorial.ipynb*  kesj/  public/
derby.log             lib/   pycharm-community-5.0.3/


In [19]:
os.chdir('lib')
%ls

aster_mgr_factory.jar*                noarch-aster-jdbc-driver.jar*
aster_to_hdfs.sh*                     QueryResult.java
com/                                  sas_mgr_factory.jar*
Impala via impyla Primer v1.0.ipynb*  sas_sqoop.sh*
KerberosLogin.jar*


In [23]:
%%writefile jdbc_hive.py
import jpype
import jaydebeapi
import os
import glob
import pandas as pd

# define username
user=os.environ['USER']

# Get hadoop jars
KERBEROS_LOGIN_JAR=r"/san-data/shared/lib/KerberosLogin.jar"
HADOOP_LIB_PATH=os.path.pathsep.join(glob.glob('%s.[jJ][aA][rR]' %
                                               r"/opt/cloudera/parcels/CDH-5.4.3-1.cdh5.4.3.p862.534/jars/*"))

#Start JVM
jpype.startJVM(jpype.getDefaultJVMPath(), "-Djava.class.path=%s:%s" % (KERBEROS_LOGIN_JAR, HADOOP_LIB_PATH))

# Kerberos Login
kerbLoginClass = jpype.JClass('org.statefarm.sr.analytics.dax.hadoop.KerberosLogin')
kerbLoginInstance = kerbLoginClass()
kerbLoginInstance.login(user+"@OPR.STATEFARM.ORG", "/home/"+user+"/"+user+".keytab")

# Establish database connection using jaydebeapi (jaydebeapi will use existing jpype jvm)
conn = jaydebeapi.connect('org.apache.hive.jdbc.HiveDriver',
                          'jdbc:hive2://da74wbedge2.opr.statefarm.org:10000/default;'
                        + 'principal=hive/da74wbedge2.opr.statefarm.org@OPR.STATEFARM.ORG')


Writing jdbc_hive.py


In [14]:
curs= conn.cursor()
curs.execute("show databases")

In [15]:
curs.fetchall()

[(u'a224_db',),
 (u'atlasid',),
 (u'claiment',),
 (u'claimsff',),
 (u'd0xy_db',),
 (u'default',),
 (u'du3z_db',),
 (u'fdwcfpoc',),
 (u'fm61_db',),
 (u'hrsepara',),
 (u'kesj_db',),
 (u'lapsecan',),
 (u'qepz_db',),
 (u'sqqq_db',),
 (u'totaloss',),
 (u'vehrepat',)]

In [26]:
curs.execute('use vehrepat')
#curs.fetchall()

In [27]:
tbl_results = conn.jconn.getMetaData().getTables(None,None,'%',None)
tbl_reader_curs = conn.cursor()
tbl_reader_curs._rs = tbl_results
read_results = tbl_reader_curs.fetchall()


AttributeError: 'NoneType' object has no attribute 'getColumnCount'

In [28]:
curs.execute('show tables')
curs.fetchall()

[(u'auto_est_sect',),
 (u'auto_est_sum',),
 (u'auto_est_sum_old',),
 (u'auto_est_team',),
 (u'auto_est_user',),
 (u'auto_rpr_fac',),
 (u'detl',),
 (u'est_party',),
 (u'est_party_detl_rltn',),
 (u'lbr_note',),
 (u'los_est',),
 (u'msg',),
 (u'non_oem',),
 (u'opt',),
 (u'p3533eeb_detl',),
 (u'rate',),
 (u'tax',),
 (u'ttl',),
 (u'y1753caaestparty',)]

In [32]:
df = pd.read_sql( "select * from detl limit 1000",conn)

In [33]:
df.head()

,detl.detl_dim_id,detl.line_prt_stts_cd,detl.line_lbr_stts_cd,detl.line_num,detl.parnt_line_num,detl.line_incld_ind,detl.line_supp_num,detl.msg_cd,detl.vndr_refr_cd,detl.line_adjst_aply_cd,...,detl.vndr_cd,detl.ver_num,detl.data_cntxt_cd,detl.fdw_rplc_ind,detl.fdw_insrt_tstmp,detl.fdw_rplc_tstmp,detl.est_ver_num,detl.src_insrt_tstmp,detl.los_est_busn_id,detl.los_est_dim_id
0,3003958965,,1,013,000,,,,001046,AINA,...,M,01,AESTMDP,N,1360155723374,253402322399999,01,1360118100000,51005805,89662175
1,3003958966,1,1,014,000,,,,005081,AINA,...,M,01,AESTMDP,N,1360155723374,253402322399999,01,1360118100000,51005805,89662175
2,3003958967,1,1,015,000,,,,001094,AINA,...,M,01,AESTMDP,N,1360155723374,253402322399999,01,1360118100000,51005805,89662175
3,3003958968,,1,016,000,,,,003650,AINA,...,M,01,AESTMDP,N,1360155723374,253402322399999,01,1360118100000,51005805,89662175
4,3003958969,,1,017,000,,,,006822,AINA,...,M,01,AESTMDP,N,1360155723374,253402322399999,01,1360118100000,51005805,89662175


In [34]:
df.isnull().any()

detl.detl_dim_id                 False
detl.line_prt_stts_cd            False
detl.line_lbr_stts_cd            False
detl.line_num                    False
detl.parnt_line_num              False
detl.line_incld_ind              False
detl.line_supp_num               False
detl.msg_cd                      False
detl.vndr_refr_cd                False
detl.line_adjst_aply_cd          False
detl.lbr_oprtn_cd                False
detl.prt_type_cd                 False
detl.lbr_type_cd                 False
detl.mtrl_type_cd                False
detl.line_adjst_type_cd          False
detl.line_adjst_pct              False
detl.line_adjst_amt              False
detl.line_desc_txt               False
detl.oem_prt_num                 False
detl.non_oem_prt_num             False
detl.est_prt_num                 False
detl.prt_clas_cd                 False
detl.suplr_refr_id               False
detl.prt_qty_cnt                 False
detl.lbr_amt                     False
detl.price_amt           

In [35]:
df.shape

(1000, 69)

In [ ]:
le = pd.read_sql('select * from los_est',conn)
le.shape

In [ ]:
le.shape

In [37]:
conn.close()
jpype.shutdownJVM()